In [1]:
# For Data Cleaning
import pandas as pd
import sqlite3

# For visualizations
import plotly.graph_objects as go
import random
import matplotlib.colors as mcolors

#To access data
import boto3
from io import StringIO

In [2]:
# Get all columns
pd.set_option('display.max_columns', None)

# Remove float calculations
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
client = boto3.client('s3')
client.download_file('pep2-yg', 'baseball/database.sqlite', 's3_baseball.sqlite')

s3_resource = boto3.resource('s3')
s3_resource.Object('pep2-yg', 'baseball/batting.csv')

temp = s3_resource.Bucket(name='pep2-yg')
for obj in temp.objects.all():
    print(obj.key)

baseball/
baseball/all_star.csv
baseball/appearances.csv
baseball/batting.csv
baseball/batting_postseason.csv
baseball/college.csv
baseball/database.sqlite
baseball/fielding.csv
baseball/fielding_outfield.csv
baseball/fielding_postseason.csv
baseball/hall_of_fame.csv
baseball/hashes.txt
baseball/home_game.csv
baseball/manager.csv
baseball/manager_award.csv
baseball/manager_award_vote.csv
baseball/manager_half.csv
baseball/park.csv
baseball/pitching.csv
baseball/pitching_postseason.csv
baseball/player.csv
baseball/player_award.csv
baseball/player_award_vote.csv
baseball/player_college.csv
baseball/postseason.csv
baseball/readme.txt
baseball/salary.csv
baseball/team.csv
baseball/team_franchise.csv
baseball/team_half.csv
nba/
yeast/


For understanding the data use this site:
https://www.baseball-almanac.com/stats4.shtml

##### Connect to sqlite file, setting up tables, combining like tables

In [ ]:
conn = sqlite3.connect('s3_baseball.sqlite')

In [ ]:
get_tables = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

cursor = conn.cursor()
cursor.execute(get_tables)

sql_names = []
for name in cursor.fetchall():
    sql_names.append(name[0])

In [ ]:
list_df = []
for name in sql_names:
    df = pd.read_sql_query(f"SELECT * FROM {name}", conn)
    list_df.append(df)

Assigning variables to each dataframe in list

In [ ]:
all_star = list_df[0]
appearances = list_df[1]
manager_award = list_df[2]
player_award = list_df[3]
manager_award_vote = list_df[4]
player_award_vote = list_df[5]
batting = list_df[6]
batting_postseason = list_df[7]
player_college = list_df[8]
fielding = list_df[9]
fielding_outfield = list_df[10]
fielding_postseason = list_df[11]
hall_of_fame = list_df[12]
home_game = list_df[13]
manager = list_df[14]
manager_half = list_df[15]
player = list_df[16]
pitching = list_df[18]
pitching_postseason = list_df[19]
postseason = list_df[22]
team = list_df[23]
team_half = list_df[25]

In [ ]:
#Part of Player
salary = list_df[20]
college = list_df[21]

#Part of Home Game
park = list_df[17]

#Part of Team
team_francise = list_df[24]

Combining Salary and Player

In [ ]:
#Used to show most recent team and salary of each player
salary_new = salary.drop_duplicates(subset=['player_id'], keep='last')
salary_new = salary[['player_id', 'team_id', 'salary']]

player = player.merge(salary_new, on='player_id', how='left')

Combining College and Player

In [ ]:
# Only used in player_college, so combined with player
merge_player_college = player_college.rename(columns={'year' : 'grad_year'}).merge(college.rename({'year' : 'grad_year'}), on='college_id')
player = merge_player_college.merge(player, on='player_id')

Combining Home Game and Park

In [ ]:
# Only used in home_game
home_game = home_game.merge(park, on='park_id')

Turning all into new .csv files

In [ ]:
# Put new tables into csv
player

,player_id,college_id,grad_year,name_full,city,state,country,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,death_month,death_day,death_country,death_state,death_city,name_first,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id,team_id,salary
0,aardsda01,pennst,2001,Pennsylvania State University,State College,PA,USA,1981,12,27,USA,CO,Denver,,,,,,,David,Aardsma,David Allan,220,75,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,SFN,300000.000
1,aardsda01,pennst,2001,Pennsylvania State University,State College,PA,USA,1981,12,27,USA,CO,Denver,,,,,,,David,Aardsma,David Allan,220,75,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,CHA,387500.000
2,aardsda01,pennst,2001,Pennsylvania State University,State College,PA,USA,1981,12,27,USA,CO,Denver,,,,,,,David,Aardsma,David Allan,220,75,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,BOS,403250.000
3,aardsda01,pennst,2001,Pennsylvania State University,State College,PA,USA,1981,12,27,USA,CO,Denver,,,,,,,David,Aardsma,David Allan,220,75,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,SEA,419000.000
4,aardsda01,pennst,2001,Pennsylvania State University,State College,PA,USA,1981,12,27,USA,CO,Denver,,,,,,,David,Aardsma,David Allan,220,75,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,SEA,2750000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47514,yatesty01,hawaiihilo,1998,University of Hawaii at Hilo,Hilo,HI,USA,1977,8,7,USA,HI,Lihue,,,,,,,Tyler,Yates,Tyler Kali,220,76,R,R,2004-04-09,2009-05-15,yatet001,yatesty01,PIT,800000.000
47515,yatesty01,hawaiihilo,1998,University of Hawaii at Hilo,Hilo,HI,USA,1977,8,7,USA,HI,Lihue,,,,,,,Tyler,Yates,Tyler Kali,220,76,R,R,2004-04-09,2009-05-15,yatet001,yatesty01,PIT,1300000.000
47516,yellela01,huntercuny,1962,"Hunter College, The City University of New York",New York,NY,USA,1943,1,4,USA,NY,Brooklyn,,,,,,,Larry,Yellen,Lawrence Alan,190,71,R,R,1963-09-26,1964-10-03,yelll101,yellela01,NaN,NaN
47517,zabelzi01,bakerks,1911,Baker University,Baldwin City,KS,USA,1891,2,18,USA,KS,Wetmore,1970,5,31,USA,WI,Beloit,Zip,Zabel,George Washington,185,73,R,R,1913-10-05,1915-09-29,zabez101,zabelzi01,NaN,NaN


##### Query 1: Average salary per team

In [ ]:
player.sort_values(by=['salary'], ascending=False)[['player_id', 'name_first', 'name_last', 'name_given', 'team_id', 'salary']]

,player_id,name_first,name_last,name_given,team_id,salary
30792,verlaju01,Justin,Verlander,Justin Brooks,DET,28000000.000
30782,verlaju01,Justin,Verlander,Justin Brooks,DET,28000000.000
30772,verlaju01,Justin,Verlander,Justin Brooks,DET,28000000.000
9317,leecl02,Cliff,Lee,Clifton Phifer,PHI,25000000.000
9318,leecl02,Cliff,Lee,Clifton Phifer,PHI,25000000.000
...,...,...,...,...,...,...
47383,withrfr01,Frank,Withrow,Frank Blaine,NaN,NaN
47384,wolveha01,Harry,Wolverton,Harry Sterling,NaN,NaN
47516,yellela01,Larry,Yellen,Lawrence Alan,NaN,NaN
47517,zabelzi01,Zip,Zabel,George Washington,NaN,NaN


In [ ]:
#Get average salary per team based on players
avg_salary = player.groupby(by='team_id')[['salary']].mean().reset_index()

#Get full team name
avg_salary = avg_salary.merge(team[['team_id', 'name']], on='team_id', how='inner')


In [ ]:
# Get rid of teams that have had name changes, keep the most recent name change
avg_salary = avg_salary.drop_duplicates(subset=['team_id', 'salary'], keep='last')[['name', 'team_id', 'salary']]

avg_salary.sort_values(by=['salary'], ascending=False)

,name,team_id,salary
1053,Los Angeles Angels of Anaheim,LAA,3862949.670
1365,New York Yankees,NYA,2944498.175
2117,Washington Nationals,WAS,2689280.050
25,Arizona Diamondbacks,ARI,2666377.575
1873,San Francisco Giants,SFN,2487500.995
1111,Los Angeles Dodgers,LAN,2433982.221
252,Boston Red Sox,BOS,2415236.435
1133,Milwaukee Brewers,MIL,2266839.082
1600,Philadelphia Phillies,PHI,2248136.284
7,Anaheim Angels,ANA,1941196.013


##### Query 2: Players who then became managers

In [ ]:
# One manager can be on different teams during their career 
manager.sort_values(by=['player_id'])

,player_id,year,team_id,league_id,inseason,g,w,l,rank,plyr_mgr
3206,actama99,2010,CLE,AL,1,162,69,93,4,N
3199,actama99,2009,WAS,NL,1,87,26,61,5,N
3242,actama99,2011,CLE,AL,1,162,80,82,2,N
3166,actama99,2008,WAS,NL,1,161,59,102,5,N
3132,actama99,2007,WAS,NL,1,162,73,89,4,N
...,...,...,...,...,...,...,...,...,...,...
2297,zimmedo01,1982,TEX,AL,1,96,38,58,6,N
2110,zimmedo01,1977,BOS,AL,1,161,97,64,3,N
2148,zimmedo01,1978,BOS,AL,1,163,99,64,2,N
2018,zimmedo01,1973,SDN,NL,1,162,60,102,6,N


In [ ]:
# Join manager table and player table on the player_ID
player_to_manager = manager[['player_id']].merge(player[['player_id', 'name_first', 'name_last', 'name_given']], on='player_id', how='inner')

In [ ]:
# Get unique managers, to get an accurate list of managers who used to be players
player_to_manager = player_to_manager.drop_duplicates()
player_to_manager.sort_values(by=['player_id'])

,player_id,name_first,name_last,name_given
973,adcocjo01,Joe,Adcock,Joseph Wilbur
669,alstowa01,Walter,Alston,Walter Emmons
1107,amalfjo01,Joey,Amalfitano,John Joseph
4931,banisje01,Jeff,Banister,Jeffery Todd
343,barryja01,Jack,Barry,John Joseph
...,...,...,...,...
4790,willima04,Matt,Williams,Matthew Derrick
1019,winklbo99,Bobby,Winkles,Bobby Brooks
288,wolveha01,Harry,Wolverton,Harry Sterling
918,yosted01,Eddie,Yost,Edward Frederick


##### Query 3: Get total number of hits, double, triples, walks, and homeruns per team per year and how much that team made

In [ ]:
team_year_stats = team[['year', 'name', 'team_id', 'h', 'double', 'triple', 'bb', 'hr']]
team_year_stats

,year,name,team_id,h,double,triple,bb,hr
0,1871,Boston Red Stockings,BS1,426,70,37,60,3
1,1871,Chicago White Stockings,CH1,323,52,21,60,10
2,1871,Cleveland Forest Citys,CL1,328,35,40,26,7
3,1871,Fort Wayne Kekiongas,FW1,178,19,8,33,2
4,1871,New York Mutuals,NY2,403,43,21,33,1
...,...,...,...,...,...,...,...,...
2800,2015,Los Angeles Dodgers,LAN,1346,263,26,563,187
2801,2015,San Francisco Giants,SFN,1486,288,39,457,136
2802,2015,Arizona Diamondbacks,ARI,1494,289,48,490,154
2803,2015,San Diego Padres,SDN,1324,260,36,426,148


In [ ]:
salary_team_q3 = salary[['year', 'team_id', 'salary']]
salary_team_q3 = salary_team_q3.groupby(by=['year', 'team_id']).sum().reset_index()
salary_team_q3

,year,team_id,salary
0,1985,ATL,14807000
1,1985,BAL,11560712
2,1985,BOS,10897560
3,1985,CAL,14427894
4,1985,CHA,9846178
...,...,...,...
883,2015,SLN,119241500
884,2015,TBA,64521233
885,2015,TEX,143742789
886,2015,TOR,112992400


In [ ]:
final_q3 = team_year_stats.merge(salary_team_q3, how='inner', on=['year', 'team_id'])
final_q3 = final_q3.rename(columns={'salary' : 'money_made'})

final_q3

,year,name,team_id,h,double,triple,bb,hr,money_made
0,1985,Atlanta Braves,ATL,1359,213,28,553,126,14807000
1,1985,Baltimore Orioles,BAL,1451,234,22,604,214,11560712
2,1985,Boston Red Sox,BOS,1615,292,31,562,162,10897560
3,1985,California Angels,CAL,1364,215,31,648,153,14427894
4,1985,Chicago White Sox,CHA,1386,247,37,471,146,9846178
...,...,...,...,...,...,...,...,...,...
883,2015,Los Angeles Dodgers,LAN,1346,263,26,563,187,215792000
884,2015,San Francisco Giants,SFN,1486,288,39,457,136,164701500
885,2015,Arizona Diamondbacks,ARI,1494,289,48,490,154,61834000
886,2015,San Diego Padres,SDN,1324,260,36,426,148,118441300


In [ ]:
batting

,player_id,year,stint,team_id,league_id,g,ab,r,h,double,triple,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,g_idp
0,abercda01,1871,1,TRO,,1,4,0,0,0,0,0,0,0,0,0,0,,,,,
1,addybo01,1871,1,RC1,,25,118,30,32,6,0,0,13,8,1,4,0,,,,,
2,allisar01,1871,1,CL1,,29,137,28,40,4,5,0,19,3,1,2,5,,,,,
3,allisdo01,1871,1,WS3,,27,133,28,44,10,2,2,27,1,1,0,2,,,,,
4,ansonca01,1871,1,RC1,,25,120,29,39,11,3,0,16,6,2,2,1,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101327,zitoba01,2015,1,OAK,AL,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
101328,zobribe01,2015,1,OAK,AL,67,235,39,63,20,2,6,33,1,1,33,26,2,0,0,3,5
101329,zobribe01,2015,2,KCA,AL,59,232,37,66,16,1,7,23,2,3,29,30,1,1,0,2,3
101330,zuninmi01,2015,1,SEA,AL,112,350,28,61,11,0,11,28,0,1,21,132,0,5,8,2,6


##### Visualization: Player Batting Career Stats

In [ ]:
# g - Games
# ab - At Bats
# r - Runs
# h - Hits
# double
# triple
# hr - home runs
# rbi - Runs Batted In
# sb - stolen bases
# cs - caught stealing

# Setting up specific columns to showcase

player_batting = batting[['player_id','ab', 'r', 'h', 'double', 'triple', 'hr', 'bb']]
player_batting = player_batting.rename(columns={'ab' : 'At Bat', 'r' : 'Runs', 'h' : 'Hits', 'double' : 'Double', 'triple' : 'Triple', 'hr' : 'Home Runs', 'bb' : 'Walks'})
player_batting


,player_id,At Bat,Runs,Hits,Double,Triple,Home Runs,Walks
0,abercda01,4,0,0,0,0,0,0
1,addybo01,118,30,32,6,0,0,4
2,allisar01,137,28,40,4,5,0,2
3,allisdo01,133,28,44,10,2,2,0
4,ansonca01,120,29,39,11,3,0,2
...,...,...,...,...,...,...,...,...
101327,zitoba01,0,0,0,0,0,0,0
101328,zobribe01,235,39,63,20,2,6,33
101329,zobribe01,232,37,66,16,1,7,29
101330,zuninmi01,350,28,61,11,0,11,21


In [ ]:
# Turning strings into ints
player_batting = player_batting.loc[player_batting['At Bat'] != '']
player_batting = player_batting.astype({'At Bat':'int', 'Runs':'int', 'Hits':'int', 'Double':'int', 'Triple':'int', 'Home Runs':'int', 'Walks':'int'})

# Getting total career batting stats

player_batting_career = player_batting.groupby(by=['player_id']).sum().reset_index()

player_batting_career = player[['player_id', 'name_given']].drop_duplicates().merge(player_batting_career, on='player_id')
player_batting_career

,player_id,name_given,At Bat,Runs,Hits,Double,Triple,Home Runs,Walks
0,aardsda01,David Allan,4,0,0,0,0,0,0
1,atherch01,Charles Morgan Herbert,242,28,60,5,6,0,21
2,baldwma01,Marcus Elmore,1155,109,188,21,18,10,82
3,bumpna01,Nathan Louis,10,1,1,0,0,0,0
4,coulsbo01,Robert Jackson,692,76,163,28,12,1,58
...,...,...,...,...,...,...,...,...,...
6243,worreti01,Timothy Howard,81,6,8,1,0,0,4
6244,worreto01,Todd Roland,27,1,2,0,1,0,1
6245,yatesty01,Tyler Kali,12,0,1,0,0,0,0
6246,yellela01,Lawrence Alan,5,0,0,0,0,0,0


In [ ]:
def random_color():
    return random.choice(list(mcolors.CSS4_COLORS))

fig = go.Figure()
fig.add_trace(
    go.Scatterpolar(
        mode='markers',
        theta=['Walks', 'Double', 'Triple', 'Home Runs'],
        fill='toself',
        marker_color='black'
    )
)

data = player_batting_career.sample()
totalVal = sum(data[['Walks', 'Double', 'Triple', 'Home Runs']].values.flatten().tolist())

while (totalVal <= 20):
    data = player_batting_career.loc[player_batting_career['player_id'] == player_batting_career.sample()[['player_id']].values[0][0]]
    totalVal = sum(data[['Walks', 'Double', 'Triple', 'Home Runs']].values.flatten().tolist())

title = f"{data[['player_id']].values[0][0]} - {data[['name_given']].values[0][0]}"
fig.update_traces(
    r = data[['Walks', 'Double', 'Triple', 'Home Runs']].values.flatten().tolist(),
    name = title,
    fillcolor=random_color()
)

fig.update_layout(
    title = {
        'text' : title,
        'x' : 0.5
    }
)
fig.show()